# DSPy in Financial Services
# Risk assessment with DSPy

DSPy makes it easy to create, evaluate, and deploy GenAI applications in a repeatable and scalable way.

In Financial Services, like in other regulated industries, it is crucial to ensure that outcomes produced by AI applications comply with laws, regulations, and ethical guidelines. In particular there are 3 aspects to consider:

1. **Explainability**: It is crucial to understand why an AI system made a particular decision or prediction. Outcomes must also be fair and non-discriminatory.
2. **Reliability**: It is important that regardless of the complexities of the processes that lead to an outcome, the elements along the way reliably function as intended.
3. **Consistency**: It is of paramount importance that outcome quality is consistent and reliable across different invocations of the system and there is no room for hallucinations.

In this notebook we will see how to use DSPy to create a risk assessment agent.

We will use the following optimizers to progress through the use case and demonstrate their differences:

- **Zero Shot**: This is the simplest form of prompting. We only provide with a description of the task (instruction).
- **Labeled Few Shot**: This is a more advanced form of prompting. We provide a description of the task (instruction), and a few labelled examples for the model to learn from and follow.
- **Bootstrap Few Shot**: This optimizer goes a level up in terms of performance by introducing the use of metrics to generate complete demonstrations of the program and optimize against them.
- **MIPRO v2**: Finally, this optimizer leverages Bayesian Optimization to optimize both instructions and few shot examples simultaneously by finding the best combination of instructions and examples.

## Zero Shot

In [1]:
import dspy
from dspy.evaluate import Evaluate
from dspy import ChainOfThought
from dsp import Claude
from typing import List, Optional
import pandas as pd
import textwrap

/home/alberto/lab/qrisk/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [54]:
import os
os.environ["DSP_CACHEBOOL"] = "False"
import phoenix as px
from phoenix.trace import using_project

phoenix_session = px.launch_app(use_temp_dir=False)
from openinference.instrumentation.dspy import DSPyInstrumentor
from openinference.instrumentation.anthropic import AnthropicInstrumentor

from phoenix.otel import register
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import ConsoleSpanExporter, SimpleSpanProcessor

endpoint = "http://127.0.0.1:6006/v1/traces"
tracer_provider = register(endpoint=endpoint, set_global_tracer_provider=False)
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

DSPyInstrumentor().instrument(tracer_provider=tracer_provider, skip_dep_check=True)
AnthropicInstrumentor().instrument(tracer_provider=tracer_provider)

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
💽 Your data is being persisted to sqlite:////home/alberto/.phoenix/phoenix.db
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix
🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://127.0.0.1:6006/v1/traces
|  Transport: HTTP
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.



In [46]:
px.active_session().use_temp_dir = False

In [53]:
px.close_app()

In [49]:
# Initialize the language model
#worker = dspy.OpenAI(model="gpt-3.5-turbo", model_type="chat", max_tokens=3000)
worker = Claude(model="claude-3-5-sonnet-20240620", max_tokens=3000)
#worker = dspy.Cohere(model="command-r-plus", max_tokens=3000)
#worker = dspy.HFModel(model = 'mistralai/Mistral-7B-Instruct-v0.2', max_tokens=3000)
dspy.configure(lm=worker)
dspy.settings.configure(backoff_time = 60)

In [50]:
# Input data
applicant_info = """
Name: John Doe
Age: 35
Annual Income: $75,000
Credit Score: 720
Existing Debts: $20,000 in student loans, $5,000 in credit card debt
Loan Amount Requested: $250,000 for a home mortgage
Employment: Software Engineer at Tech Corp for 5 years
"""

In [51]:
class ZeroShot(dspy.Module):
    """
    You are given a piece of text that contains information about an applicant. 
    Analyze the applicant's financial information and return a risk assessment.
    """
    def __init__(self):
        super().__init__()
        self.prog = dspy.Predict("question -> answer")

    def forward(self, applicant):
        return self.prog(question="Analyze the applicant's financial information and return a risk assessment. Applicant: " + applicant)

In [52]:
with using_project("zero_shot"):
    module = ZeroShot()
    response = module(applicant_info)
    print(f"ZeroShot response:\n {response}")

    prompt_used = worker.inspect_history(n=1)
    print(f"Prompt used: {prompt_used}")

ZeroShot response:
 Prediction(
    answer="Based on the provided financial information for John Doe, here's a risk assessment:\n\nAnswer: Low to Moderate Risk\n\nReasons for this assessment:\n\n1. Positive factors:\n   - Stable employment: 5 years at Tech Corp as a Software Engineer\n   - Good credit score: 720 is considered good and indicates responsible credit management\n   - Solid annual income: $75,000 is a respectable salary for a 35-year-old professional\n   - Debt-to-income ratio: Existing debts are relatively low compared to annual income\n\n2. Potential concerns:\n   - Loan amount requested ($250,000) is more than 3 times the annual income, which is on the higher side\n   - Existing debts, while not excessive, do add to the overall financial obligations\n\n3. Recommendations:\n   - Verify the applicant's ability to make consistent mortgage payments\n   - Consider the local housing market and ensure the loan amount is appropriate for the property value\n   - Review the applic

## Labeled Few Shot

In [55]:
import json
from dspy.teleprompt import LabeledFewShot

In [56]:
class RiskAssessment(dspy.Signature):
    """Analyze the applicant's financial information and return a risk assessment."""
    question = dspy.InputField()
    applicant = dspy.InputField()
    answer = dspy.OutputField(desc="""
                              A thorough risk analysis about the applicant, justifying the assessment 
                              for each of the parameters considered from the applicant
                              """
                              )

In [57]:
class RiskAssessmentAgent(dspy.Module):
    def __init__(self):
        self.question = "Analyze the applicant's financial information and return a risk assessment."
        self.assess_risk = ChainOfThought(RiskAssessment, n=3)
    def forward(self, applicant:str):
        question = self.question
        applicant = applicant
        pred = self.assess_risk(question=question, applicant=applicant)

        return dspy.Prediction(answer = pred.answer)

In [58]:
# Load the training data
dataset = json.load(open("data/training_data.json", "r"))['examples']
trainset = [dspy.Example(question="Analyze the applicant's financial information and return a risk assessment", 
                         applicant=e['applicant'], 
                         answer=e['answer']) for e in dataset]

In [59]:
# Train
teleprompter = LabeledFewShot()
lfs_optimized_advisor = teleprompter.compile(RiskAssessmentAgent(), 
                                             trainset=trainset[3:]
                                            )

In [60]:
with using_project("labeled_few_shot"):
    response = lfs_optimized_advisor(applicant_info)
    wrapped_response = textwrap.fill(response.answer, width=70)
    print(f"LabeledFewShot Optimised response:\n {wrapped_response}")

LabeledFewShot Optimised response:
 Based on the analysis of John Doe's financial information, the risk
assessment is Low to Medium Risk. Here's the justification:  1.
Positive Factors:    - Good credit score (720) indicating responsible
credit management.    - Stable employment in a well-paying field.    -
Reasonable loan-to-income ratio for a mortgage.  2. Potential
Concerns:    - Existing debt, while not excessive, does add to his
overall financial obligations.    - The new debt-to-income ratio would
be on the higher side at 52.42%, which could strain his budget.  3.
Overall Assessment:    John Doe appears to be a responsible borrower
with a stable job and good credit history. The mortgage amount
requested is in line with his income, although it will significantly
increase his debt load. The main risk factor is the potentially high
debt-to-income ratio after taking on the mortgage.  Recommendation:
The lender should feel relatively confident in John's ability to repay
the loan, but 

In [61]:
prompt_used = worker.inspect_history(n=1)
print(f"Prompt used: {prompt_used}")




Analyze the applicant's financial information and return a risk assessment.

---

Follow the following format.

Question: ${question}

Applicant: ${applicant}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: A thorough risk analysis about the applicant, justifying the assessment for each of the parameters considered from the applicant

---

Question: Analyze the applicant's financial information and return a risk assessment
Applicant: Name: Richard Turner Age: 45 Annual Income: $95,000 Credit Score: 620 Existing Debts: $20,000 in credit card debt Loan Amount Requested: $30,000 for home renovations Employment: Engineer at a manufacturing company for 15 years
Answer: Based on the analysis of Richard Turner's financial information, the risk assessment is Medium. Here's the justification for this assessment: 1. Income: Richard's annual income of $95,000 is relatively high, providing a good foundation for loan repayment. 2. Employment Stability: 15 

In [77]:
lfs_optimized_advisor.save('lfs_optimized_advisor_compiled.json')

[('assess_risk', Predict(StringSignature(question, applicant -> rationale, answer
    instructions="Analyze the applicant's financial information and return a risk assessment."
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    applicant = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Applicant:', 'desc': '${applicant}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': '\n                              A thorough risk analysis about the applicant, justifying the assessment \n                              for each of the parameters considered from the applicant\n                              ', '__dspy_field_type': 

## Bootstrap Few Shot

In [62]:
from dspy.teleprompt import BootstrapFewShot

In [63]:
# Define the signature for automatic assessments.
class Assess(dspy.Signature):
    """Assess the quality of a risk assessment along the specified dimension."""

    assessed_text = dspy.InputField()
    assessment_question = dspy.InputField()
    assessment_answer = dspy.OutputField(desc="Yes or No")

In [64]:
def risk_assessment_metric(gold, pred, trace=None):
    applicant, risk_assessment = gold.applicant, pred.answer

    correct = f"The text above should provide a risk assessment for `{applicant}`. Does it do so? Answer with Yes or No."
    complete = f"Does the text above make for a reasoned assessment across all areas mentioned in `{applicant}`? Answer with Yes or No."

    with dspy.context(lm=worker):
        correct =  dspy.Predict(Assess)(assessed_text=risk_assessment, assessment_question=correct)
        complete = dspy.Predict(Assess)(assessed_text=risk_assessment, assessment_question=complete)

    correct, complete = [m.assessment_answer.split()[0].lower() == 'yes' for m in [correct, complete]]
    score = (correct + complete) if correct else 0

    if trace is not None: return score >= 2
    return score / 2.0

In [17]:
# Load the training data
dataset = json.load(open("data/training_data.json", "r"))['examples']
trainset = [dspy.Example(question="Analyze the applicant's financial information and return a risk assessment", 
                         applicant=e['applicant'], 
                         answer=e['answer']) for e in dataset]

In [18]:
with using_project("bootstrap_few_shot"):
    bfs_trainset = [x.with_inputs('applicant') for x in trainset]
    config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)
    bfs_optimized = BootstrapFewShot(metric=risk_assessment_metric, **config)
    bfs_optimized_advisor = bfs_optimized.compile(RiskAssessmentAgent(),
                                                trainset=bfs_trainset)

 56%|█████▌    | 15/27 [10:07<08:05, 40.47s/it]

Bootstrapped 4 full traces after 16 examples in round 0.


In [19]:
with using_project("bootstrap_few_shot"):
    response = bfs_optimized_advisor(applicant_info)
    wrapped_response = textwrap.fill(response.answer, width=70)
    print(f"BootstrapFewShot Optimised response:\n {wrapped_response}")

BootstrapFewShot Optimised response:
 Based on the analysis of John Doe's financial information, the risk
assessment is Medium Risk. Here's the justification for this
assessment:  1. Income and Employment: John's stable job as a Software
Engineer with 5 years of experience and a solid annual income of
$75,000 provide a good foundation for loan repayment.  2. Credit
Score: His credit score of 720 is good, indicating responsible credit
management and reducing the risk of default.  3. Existing Debts: While
the student loan debt is significant, it's not unusual for his
profession. The credit card debt, however, suggests some potential for
financial strain.  4. Debt-to-Income Ratio: The current DTI of 33.33%
is manageable, but the potential DTI after the mortgage (51.30%) is
concerning. Most lenders prefer a DTI below 43% for qualified
mortgages.  5. Housing Expense Ratio: At 17.97%, this is within the
generally acceptable range of 28% or less.  6. Loan Amount: The
$250,000 mortgage is subs

In [20]:
prompt_used = worker.inspect_history(n=1)
print(f"Prompt used: {prompt_used}")




Analyze the applicant's financial information and return a risk assessment.

---

Follow the following format.

Question: ${question}

Applicant: ${applicant}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: A thorough risk analysis about the applicant, justifying the assessment for each of the parameters considered from the applicant

---

Question: Analyze the applicant's financial information and return a risk assessment.

Applicant: Name: John Smith Age: 35 Annual Income: $80,000 Credit Score: 750 Existing Debts: $0 Loan Amount Requested: $50,000 for a car loan Employment: Software Engineer at XYZ Corp for 5 years

Reasoning: Let's think step by step in order to Reasoning: Let's think step by step in order to produce a comprehensive risk assessment for John Smith: 1. Age: At 35, John is in his prime working years, which is a positive factor. 2. Annual Income: $80,000 is a solid income, indicating good earning potential and ability to repay.

In [21]:
bfs_optimized_advisor.save('bfs_optimized_advisor_compiled.json')

[('assess_risk', Predict(StringSignature(question, applicant -> rationale, answer
    instructions="Analyze the applicant's financial information and return a risk assessment."
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    applicant = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Applicant:', 'desc': '${applicant}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': '\n                              A thorough risk analysis about the applicant, justifying the assessment \n                              for each of the parameters considered from the applicant\n                              ', '__dspy_field_type': 

## Bootstrap Few Shot with Advanced Metrics using G-Eval (https://arxiv.org/abs/2303.16634)

In [65]:
from deepeval.metrics import GEval, BiasMetric, AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from openinference.instrumentation.openai import OpenAIInstrumentor

OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)


In [66]:
def risk_assessment_metric_adv(gold, pred, trace=None):
    applicant, risk_assessment = gold.applicant, pred.answer

    test_case = LLMTestCase(
            input=applicant,
            actual_output=risk_assessment
    )
    bias_metric = BiasMetric(
        threshold=0.7,
        include_reason=True,
        async_mode=False
    )
    bias_metric.measure(test_case)
    answer_relevancy_metric = AnswerRelevancyMetric(
        threshold=0.7,
        include_reason=True,
        async_mode=False
    )
    answer_relevancy_metric.measure(test_case)
    coherence_metric = GEval(
        name="coherence",
        criteria="Coherence - determine if the actual output is coherent with the input",
        evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
        evaluation_steps=[
            "Check if the actual output is logically consistent with the input on the basis of the instructions from the input",
            "Assess if the actual output is relevant to the input and maintains a logical flow on the basis of the instructions from the input",
        ],
        async_mode=False
    )
    coherence_metric.measure(test_case)
    
    
    score = (1 - bias_metric.score) + answer_relevancy_metric.score + coherence_metric.score
    
    if trace is not None: return score >= 3
    return score / 3.0

In [67]:
# Load the training data
dataset = json.load(open("data/training_data.json", "r"))['examples']
trainset = [dspy.Example(question="Analyze the applicant's financial information and return a risk assessment", 
                         applicant=e['applicant'], 
                         answer=e['answer']) for e in dataset]

In [68]:
with using_project("bootstrap_few_shot_adv_metrics"):
    bfs_trainset = [x.with_inputs('applicant') for x in trainset]
    config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)
    bfs_optimized_adv_metrics = BootstrapFewShot(metric=risk_assessment_metric_adv, **config)
    bfs_optimized_advisor_adv_metrics = bfs_optimized_adv_metrics.compile(RiskAssessmentAgent(),
                                                trainset=bfs_trainset)

  0%|          | 0/27 [00:00<?, ?it/s]

/home/alberto/lab/qrisk/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/home/alberto/lab/qrisk/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/home/alberto/lab/qrisk/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/home/alberto/lab/qrisk/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

...

/home/alberto/lab/qrisk/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/home/alberto/lab/qrisk/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/home/alberto/lab/qrisk/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

100%|██████████| 27/27 [21:52<00:00, 48.62s/it]

Bootstrapped 1 full traces after 27 examples in round 0.


In [69]:
with using_project("bootstrap_few_shot_adv_metrics"):
    response = bfs_optimized_advisor_adv_metrics(applicant_info)
    wrapped_response = textwrap.fill(response.answer, width=70)
    print(f"BootstrapFewShot Optimised with Advanced Metricsresponse:\n {wrapped_response}")

BootstrapFewShot Optimised with Advanced Metricsresponse:
 Based on the analysis of John Doe's financial information, the risk
assessment is Medium. Here's the justification for this assessment:
1. Income and Employment: John's annual income of $75,000 as a
Software Engineer, with 5 years at the same company, indicates
stability and a good foundation for loan repayment. However, the
income is somewhat low for the size of the mortgage requested.  2.
Credit History: His good credit score of 720 suggests a history of
responsible credit management, which reduces the risk of default.  3.
Existing Debts: The combination of student loans and credit card debt
is manageable but does add to his overall financial obligations.  4.
Debt-to-Income Ratio: The estimated post-mortgage DTI of 40-45% is at
the upper limit of what most lenders consider acceptable, which
increases the risk.  5. Loan Amount and Purpose: While a home mortgage
is a common and justified loan purpose, the $250,000 amount is mor

In [70]:
prompt_used = worker.inspect_history(n=1)
print(f"Prompt used: {prompt_used}")




Analyze the applicant's financial information and return a risk assessment.

---

Follow the following format.

Question: ${question}

Applicant: ${applicant}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: A thorough risk analysis about the applicant, justifying the assessment for each of the parameters considered from the applicant

---

Question: Analyze the applicant's financial information and return a risk assessment.

Applicant: Name: John Smith Age: 35 Annual Income: $80,000 Credit Score: 750 Existing Debts: $0 Loan Amount Requested: $50,000 for a car loan Employment: Software Engineer at XYZ Corp for 5 years

Reasoning: Let's think step by step in order to Reasoning: Let's think step by step in order to produce a comprehensive risk assessment for John Smith: 1. Age: At 35, John is in his prime working years, which is a positive factor. 2. Annual Income: $80,000 is a solid income, indicating good earning potential and ability to repay.

In [71]:
bfs_optimized_advisor_adv_metrics.save('bfs_optimized_advisor_with_adv_metrics_compiled.json')

[('assess_risk', Predict(StringSignature(question, applicant -> rationale, answer
    instructions="Analyze the applicant's financial information and return a risk assessment."
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    applicant = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Applicant:', 'desc': '${applicant}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': '\n                              A thorough risk analysis about the applicant, justifying the assessment \n                              for each of the parameters considered from the applicant\n                              ', '__dspy_field_type': 

## MIPRO

In [72]:
from dspy.teleprompt import MIPROv2

In [73]:
with using_project("mipro_v2_adv_metrics"):
    worker_gpt4o = dspy.OpenAI(model="gpt-4o", model_type="chat", max_tokens=5000)
    dspy.configure(lm=worker_gpt4o)
    mipro_trainset = [x.with_inputs('applicant') for x in trainset]
    config = dict(num_candidates=10, init_temperature=0.1)
    mipro_optimized = MIPROv2(metric=risk_assessment_metric_adv, **config)
    mipro_optimized_advisor = mipro_optimized.compile(RiskAssessmentAgent(),
                                                trainset=mipro_trainset,
                                                max_bootstrapped_demos=2,
                                                max_labeled_demos=2,
                                                num_trials=5,
                                                minibatch=False
                                                )
    mipro_optimized_advisor.save(f'mipro_optimized_advisor_compiled.json')
    response = mipro_optimized_advisor(applicant_info)
    print(f"MIPRO Optimised response:\n {response}")
    

Projected Language Model (LM) Calls

Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:


- Prompt Model: 10 data summarizer calls + 10 * 1 lm calls in program + (2) lm calls in program aware proposer = 22 prompt model calls
- Task Model: 21 examples in val set * 5 batches * # of LM calls in your program = (30 * # of LM calls in your program) task model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the proje

  0%|          | 0/6 [00:00<?, ?it/s]

/home/alberto/lab/qrisk/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/home/alberto/lab/qrisk/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/home/alberto/lab/qrisk/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

...

/home/alberto/lab/qrisk/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/home/alberto/lab/qrisk/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Average Metric: 19.77194159610227 / 20  (98.9):  95%|█████████▌| 20/21 [00:54<00:02,  3.00s/it]/home/alberto/lab/qrisk/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


Average Metric: 20.75935690745567 / 21  (98.9): 100%|██████████| 21/21 [00:56<00:00,  2.68s/it]


Score: 98.85 with parameters ['Predictor 1: Instruction 3', 'Predictor 1: Few-Shot Set 8'].
New best score updated! Score: 98.85 on trial 5.


[('assess_risk', Predict(StringSignature(question, applicant -> rationale, answer
    instructions="Analyze the applicant's financial information and return a risk assessment."
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    applicant = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Applicant:', 'desc': '${applicant}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': '\n                              A thorough risk analysis about the applicant, justifying the assessment \

In [74]:
response = textwrap.fill(response.answer, width=70)
print(f"MIPRO Optimised response:\n {wrapped_response}")

MIPRO Optimised response:
 Based on the analysis of John Doe's financial information, the risk
assessment is Medium. Here's the justification for this assessment:
1. Income and Employment: John's annual income of $75,000 as a
Software Engineer, with 5 years at the same company, indicates
stability and a good foundation for loan repayment. However, the
income is somewhat low for the size of the mortgage requested.  2.
Credit History: His good credit score of 720 suggests a history of
responsible credit management, which reduces the risk of default.  3.
Existing Debts: The combination of student loans and credit card debt
is manageable but does add to his overall financial obligations.  4.
Debt-to-Income Ratio: The estimated post-mortgage DTI of 40-45% is at
the upper limit of what most lenders consider acceptable, which
increases the risk.  5. Loan Amount and Purpose: While a home mortgage
is a common and justified loan purpose, the $250,000 amount is more
than 3 times John's annual inc

In [75]:
prompt_used = worker_gpt4o.inspect_history(n=1)
print(f"Prompt used: {prompt_used}")




Analyze the provided financial information of the applicant and deliver a comprehensive risk assessment, including a detailed rationale for each parameter considered.

---

Follow the following format.

Question: ${question}

Applicant: ${applicant}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: A thorough risk analysis about the applicant, justifying the assessment for each of the parameters considered from the applicant

---

Question: Analyze the applicant's financial information and return a risk assessment.

Applicant: Name: John Smith Age: 35 Annual Income: $80,000 Credit Score: 750 Existing Debts: $0 Loan Amount Requested: $50,000 for a car loan Employment: Software Engineer at XYZ Corp for 5 years

Reasoning: Let's think step by step in order to produce the answer. We start by evaluating each aspect of John Smith's financial profile to determine the level of risk associated with lending him the requested car loan. 1. **Age (35)**: Joh

In [76]:
mipro_optimized_advisor.save(f'mipro_optimized_advisor_compiled.json')

[('assess_risk', Predict(StringSignature(question, applicant -> rationale, answer
    instructions="Analyze the applicant's financial information and return a risk assessment."
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    applicant = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Applicant:', 'desc': '${applicant}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': '\n                              A thorough risk analysis about the applicant, justifying the assessment \n                              for each of the parameters considered from the applicant\n                              ', '__dspy_field_type': 

In [38]:
mipro_optimized_advisor.load(f'mipro_optimized_advisor_compiled.json')
response = mipro_optimized_advisor(applicant_info)
print(f"MIPRO Optimised response:\n {response}")
wrapped_response = textwrap.fill(response.answer, width=70)
prompt_used = worker_gpt4o.inspect_history(n=1)
print(f"Prompt used: {prompt_used}")

MIPRO Optimised response:
 Prediction(
    answer="Based on the analysis of John Doe's financial information, the risk assessment is Medium-High Risk. Here's the justification for this assessment:\n\n1. **Income and Employment**: John's stable income and employment in a solid industry are positive factors. However, his income may not be sufficient to comfortably manage the requested mortgage in addition to existing debts.\n\n2. **Credit Worthiness**: A credit score of 720 is good, indicating a history of managing credit responsibly. This reduces the risk of default.\n\n3. **Debt Management**: The $20,000 in student loans is typical and low-risk, but the $5,000 in credit card debt requires careful management due to potentially high interest rates.\n\n4. **Debt-to-Income Ratio**: The projected DTI after the mortgage is concerning, as it far exceeds the generally accepted threshold, indicating potential financial strain.\n\n5. **Loan Amount**: The requested mortgage amount is high relativ